In [19]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import requests


In [22]:
url="https://arena.kakaocdn.net/kakao_arena/melon_autoplaylist/train.json?TWGServiceId=kakao_arena&Expires=1625678525&Signature=M4CftdRNvlwfiQvoeBVE6GE6r20%3D&AllowedIp=1.246.85.102&download"
r=requests.get(url)
r.encoding="utf-8"
train=r.json()
train_df=pd.DataFrame(train)
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


In [23]:
url="https://arena.kakaocdn.net/kakao_arena/melon_autoplaylist/genre_gn_all.json?TWGServiceId=kakao_arena&Expires=1625678543&Signature=2%2BpY86TVtuZ187k1v6oaPoT9eLY%3D&AllowedIp=1.246.85.102&download"
r=requests.get(url)
r.encoding="utf-8"
genre=r.json()
genre_ser=pd.Series(genre)
genre_ser

GN0100       발라드
GN0101    세부장르전체
GN0102       '80
GN0103       '90
GN0104       '00
           ...  
GN2900       뮤지컬
GN2901    세부장르전체
GN2902     국내뮤지컬
GN2903     국외뮤지컬
GN3000     크리스마스
Length: 254, dtype: object

In [24]:
url="https://arena.kakaocdn.net/kakao_arena/melon_autoplaylist/song_meta.json?TWGServiceId=kakao_arena&Expires=1625678557&Signature=frq66%2BpQLWBgcG3RTivdtaBen3Y%3D&AllowedIp=1.246.85.102&download"
r=requests.get(url)
r.encoding="utf-8"
song=r.json()
song_df=pd.DataFrame(song)
song_df.head(3)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2


In [25]:
import KerasModel

train_df=pd.read_csv("./keras_train.csv")
# 모델 적용 전 전처리
train_df[["t1","t2","t3"]]=train_df[["t1","t2","t3"]].fillna("-")
# 임베딩 전 어휘목록 크기 생성
songs_size=len(song)

gr_nq=list(genre)

t1_nq=train_df["t1"].unique()
t2_nq=train_df["t2"].unique()
t3_nq=train_df["t3"].unique()
tg_nq=np.concatenate((t1_nq,t2_nq,t3_nq))
tg_nq=np.unique(tg_nq)

print(tg_nq.shape)

# 모델 적용
model=KerasModel.get_model(songs_size,tg_nq,gr_nq)

(23653,)
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tags (InputLayer)               [(None, 3)]          0                                            
__________________________________________________________________________________________________
genrs (InputLayer)              [(None, 5)]          0                                            
__________________________________________________________________________________________________
songs (InputLayer)              [(None, 6)]          0                                            
__________________________________________________________________________________________________
string_lookup_4 (StringLookup)  (None, 3)            0           tags[0][0]                       
___________________________________________________________________________________

In [26]:
train_df=pd.read_csv("./keras_trainTF.csv")
train_df[["t1","t2","t3"]]=train_df[["t1","t2","t3"]].fillna("-")
sg_size=len(song)
gr_nq=list(genre)
t1_nq=train_df["t1"].unique()
t2_nq=train_df["t2"].unique()
t3_nq=train_df["t3"].unique()
tg_nq=np.concatenate((t1_nq,t2_nq,t3_nq))
tg_nq=np.unique(tg_nq)

In [27]:
import KerasModel
model1=KerasModel.rank_model(sg_size,tg_nq,gr_nq)

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tags (InputLayer)               [(None, 3)]          0                                            
__________________________________________________________________________________________________
genres (InputLayer)             [(None, 5)]          0                                            
__________________________________________________________________________________________________
songs (InputLayer)              [(None, 6)]          0                                            
__________________________________________________________________________________________________
string_lookup_6 (StringLookup)  (None, 3)            0           tags[0][0]                       
____________________________________________________________________________________________

In [28]:
import KerasModel
# KerasModel 모델 훈련 - 스펙트로그램으로 곡들의 유사성을 예측하고 곡을 추천해주는 model????????
model2=KerasModel.image_sim()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img1 (InputLayer)               [(None, 48, 200, 1)] 0                                            
__________________________________________________________________________________________________
img2 (InputLayer)               [(None, 48, 200, 1)] 0                                            
__________________________________________________________________________________________________
model_6 (Functional)            (None, 32)           8846432     img1[0][0]                       
                                                                 img2[0][0]                       
__________________________________________________________________________________________________
dot_similarity (Dot)            (None, 1)            0           model_6[0][0]              

In [31]:
model1.load_weights("C:/Users/hyebin/Desktop/lee/Personal_Project/Kakao_Melon_Playlist/model/searchover")


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002CC90CBE340> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002CC908F5940>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002CC9115B2E0> and <tensorflow.python.keras.layers.core.Dense object at 0x000002CC90CBE340>).


ValueError: Tensor's shape (255, 50) is not compatible with supplied shape (23667, 50)

In [13]:
import random

model1.load_weights("./model/search")
model2.load_weights("./model/rank")

songs_map={s["id"]: s for s in song}

for pl in train[:30]:
    songs=pl["songs"]
    tags=pl["tags"]
    gnrs=[]
    for s in songs:
        gnr=songs_map[s]["song_gn_dtl_gnr_basket"]
        gnrs.extend(gnr)
        
    gnrs=set(gnrs)
    
    if len(gnrs)<5 or len(songs)<7:
        continue
        
    sg_test=random.sample(songs,6)
    gr_test=random.sample(gnrs,5)
    tg_test=["-","-","-"]
    if len(tags)>3:
        tg_test=random.sample(tags,3)
    else:
        for i,t in enumerate(tags):
            tf_test[i]=t
    tg_test=np.array(tg_test)
    
    # 
    psongs=model1.predict({"songs":np.array([sg_test]),"tags":np.array([tg_test]),"genres":np.array([gr_test])})
    
    candidate=np.argsort(psongs[0])[-10:]
    print(psongs[0][candidate])
    
    socre=np.zeros((10,))
    for i,cs in enumerate(candidate):
        psongs=model2.predict({"songs":np.array([sg_test]),"tags":np.array([tg_test]),
                               "genres":np.array([gr_test]),"target":np.array([cs])})
        score[i]=psongs[0]
    
    for sg in song_test:
        for i,cs in enumerate(candidate):
            
            psongs=model2.predict(genre)
            score[i]=psongs[0]

        
    top5=np.argsort(score)[-5:]
    print(score[top5])
    print(candidate[top5])
    acc=[p for p in candidate[top3] if p in songs]
    print(acc)


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002CC91DC7DF0> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002CC938C0EB0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002CC91A36310> and <tensorflow.python.keras.layers.core.Dense object at 0x000002CC91DC7DF0>).


ValueError: Tensor's shape (255, 50) is not compatible with supplied shape (23667, 50)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
mel0=np.load("./arena_mel/0/3.npy")
# convolution layer 는 3차원이 들어가서 차원수 변경
mel0=np.expand_dims(mel0,axis=-1)
print(mel0.shape)

plt.imshow(mel0,cmap="gray")
plt.show()

# 이미지의 길이가 길어서 부분만 가져와 접근해보기

In [ ]:
import KerasModel

model=KerasModel.image_sim()

In [ ]:
import tensorflow as tf
import pandas as pd
import KerasModel

train_df=pd.read_csv("./keras_trainTF.csv")

re_df=train_df[train_df["s1"]<30000]
re_df=re_df[re_df["target"]<30000]
re_df=re_df.rename({"s1":"input"},axis=1)

dg=KerasModel.DataGenerator(re_df,32)

for image_batch,labels_batch in dg:
    print(image_batch)
    print(labels_batch)
    break